In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from statsmodels.stats.proportion import proportion_confint, samplesize_confint_proportion

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X ∼ N(μ,σ2) лежит в интервале μ ± c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки. 

In [2]:
round(norm.ppf(1 - (1 - 0.997) / 2), 4)

2.9677

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
proba_placebo = 189 / 11034
proba_aspirin = 104 / 11037
print(round(proba_placebo - proba_aspirin, 4))

0.0077


Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки. 

In [4]:
def proportions_confint_diff_ind(a, b, n1, n2, alpha = 0.05):    
    z = norm.ppf(1 - alpha / 2.)   
    p1 = a / n1
    p2 = b / n2
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [5]:
round(proportions_confint_diff_ind(189, 104, 11034, 11037)[1], 4)

0.0107

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки. 

In [6]:
odds_placebo = (189/11034) / (1 - (189/11034))
odds_aspirin = (104/11037) / (1 - (104/11037))
print(round(odds_placebo/odds_aspirin, 4))

1.8321


Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

 - составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
 - установите random seed=0;
 - сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [7]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [8]:
def get_odds(proportion):
    return (sum(proportion)/len(proportion)) / (1 - (sum(proportion)/len(proportion)))

In [9]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [10]:
np.random.seed(0)

placebo_data = np.array([1]*189 + [0]*(11034-189))
aspirin_data = np.array([1]*104 + [0]*(11037-104))

placebo_odds = list(map(get_odds, get_bootstrap_samples(placebo_data, 1000)))
aspirin_odds = list(map(get_odds, get_bootstrap_samples(aspirin_data, 1000)))

odds_ratio = [odds_p / odds_a for odds_p, odds_a in zip(placebo_odds, aspirin_odds)]

In [11]:
round(stat_intervals(odds_ratio, 0.05)[0], 4)

1.4629